In [2]:
import pandas as pd     
import matplotlib.pyplot as plt

In [3]:
df_cp = pd.read_csv("dataset_with_timestamp")
df_cp = df_cp.drop(['hour', 'month', 'year', 'day'], axis=1)

In [4]:
df_cp['timestamp'] = pd.to_datetime(df_cp['timestamp'], format='%Y-%m-%d %H:%M:%S')


In [5]:
df_cp = df_cp.groupby(["timestamp","country"]).mean().reset_index()
df_cp = df_cp.set_axis(df_cp.timestamp)
df_cp = df_cp.drop(["timestamp"], axis=1)

In [6]:
df_indices = pd.read_csv("daily_indices_82_to_19.csv")
df_indices['timestamp'] = pd.to_datetime(df_indices['timestamp'], format='%Y-%m-%d')
df_indices = df_indices[["timestamp","nao", "nino34", "ao"]]
df_indices = df_indices.set_axis(df_indices.timestamp)
df_indices = df_indices.drop(["timestamp"], axis=1)





In [7]:
def normalize(df, how="min-max"):
    if how == "min_max" :
        return (df - df.min())/(df.max()-df.min())
    if how == "mean" :
        return (df-df.mean())/df.std()



In [31]:
def get_cp_indices_df(df_cp, df_indices, selected_coutry = None, data_resolution = "M") :


    if not selected_coutry : selected_coutry = df_cp.country.unique()

    df_cp_filter = df_cp[df_cp.country.isin(selected_coutry) ]
    df_cp_filter = df_cp_filter.groupby([df_cp_filter.index.to_period(data_resolution)]).mean().reset_index()
    df_cp_filter = df_cp_filter.set_index(df_cp_filter.timestamp).drop(["timestamp"], axis=1)



    df_indices_filter = df_indices.groupby([df_indices.index.to_period(data_resolution)]).mean().reset_index()
    df_indices_filter = df_indices_filter.set_index(df_indices_filter.timestamp).drop(["timestamp"], axis=1)


    df_indices_filter = normalize(df_indices_filter, how="mean")
    df_cp_filter.capacity_factor = normalize(df_cp_filter.capacity_factor, how="mean")

    df_cp_indices = df_cp_filter.join(df_indices_filter)

    return df_cp_indices

    

def compare_cp_index(df_cp_indices, smooth = 6, indices = None, start=1990, end=2019, show_plot = True):
    if indices : 
        df_cp_indices = df_cp_indices[indices+["capacity_factor"]] 
    df_cp_indices = df_cp_indices[(df_cp_indices.index.year > start) & (df_cp_indices.index.year < end)]
    df_cp_indices = df_cp_indices.rolling(smooth, center=True).mean()
    
    if show_plot : df_cp_indices.plot(figsize=(20,10))
    
    return df_cp_indices.corr().capacity_factor
 









In [32]:
corr = dict()
for selected_country in df_cp.country.unique() :
    df_cp_indices = get_cp_indices_df(df_cp, df_indices, selected_coutry=[selected_country])
    corr[selected_country] = compare_cp_index(df_cp_indices, indices=["nao"], start = 1990, show_plot= False)["nao"]


C:\Users\Alban\AppData\Local\Temp\ipykernel_20364\210496198.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_cp_filter = df_cp_filter.groupby([df_cp_filter.index.to_period(data_resolution)]).mean().reset_index()
C:\Users\Alban\AppData\Local\Temp\ipykernel_20364\210496198.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_cp_filter = df_cp_filter.groupby([df_cp_filter.index.to_period(data_resolution)]).mean().reset_index()
C:\Users\Alban\AppData\Local\Temp\ipykernel_20364\210496198.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, nu

In [33]:
dict(sorted(corr.items(), key=lambda item: item[1])[::-1])

{'IE': 0.497551919105371,
 'NO': 0.48966756031794023,
 'GB': 0.4783587327173552,
 'SE': 0.46613097787351493,
 'FI': 0.4550933392400904,
 'CZ': 0.42883171383907975,
 'DE': 0.4254985033654943,
 'PL': 0.4098411573298391,
 'SK': 0.4048360942268869,
 'AT': 0.4012089301629372,
 'DK': 0.3929787400477672,
 'NL': 0.3850794639957258,
 'LT': 0.37981257593935885,
 'LV': 0.37154786268273016,
 'CH': 0.35820622485962794,
 'BE': 0.3574535867591478,
 'LU': 0.3384334981262021,
 'RO': 0.33466507358409037,
 'FR': 0.3265600944824543,
 'BG': 0.3177197528694933,
 'HU': 0.3092285207665751,
 'ME': 0.28145732395211454,
 'HR': 0.2520786442341703,
 'IT': 0.24460186720282978,
 'GR': 0.14712866626563129,
 'ES': 0.14450530554525023,
 'SI': 0.13438264088370092,
 'PT': 0.0839553177606717}